In [1]:
from pymilvus import MilvusClient

host = "localhost"
port = "19530"

milvus_client = MilvusClient(host=host, port=port)

In [2]:
from pymilvus import FieldSchema, DataType, CollectionSchema

VECTOR_LENGTH = 768  # check the dimensionality for Silver Retriever Base (v1.1) model

id_field = FieldSchema(
    name="id", dtype=DataType.INT64, is_primary=True, description="Primary id"
)
text = FieldSchema(
    name="text", dtype=DataType.VARCHAR, max_length=4096, description="Page text"
)
embedding_text = FieldSchema(
    "embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=VECTOR_LENGTH,
    description="Embedded text",
)

fields = [id_field, text, embedding_text]

schema = CollectionSchema(
    fields=fields,
    auto_id=True,
    enable_dynamic_field=True,
    description="RAG Texts collection",
)

In [3]:
COLLECTION_NAME = "rag_texts_and_embeddings"

milvus_client.create_collection(collection_name=COLLECTION_NAME, schema=schema)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="L2",
    params={"M": 4, "efConstruction": 64},  # lower values for speed
)

milvus_client.create_index(collection_name=COLLECTION_NAME, index_params=index_params)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME))

['rag_texts_and_embeddings']
{'collection_name': 'rag_texts_and_embeddings', 'auto_id': True, 'num_shards': 1, 'description': 'RAG Texts collection', 'fields': [{'field_id': 100, 'name': 'id', 'description': 'Primary id', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'text', 'description': 'Page text', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096}}, {'field_id': 102, 'name': 'embedding', 'description': 'Embedded text', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'functions': [], 'aliases': [], 'collection_id': 461836471237098274, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 461837262024343558}


In [4]:
# define data source and destination
## the document origin destination from which document will be downloaded
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the document
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the processed document
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"

## local destination of the embedded pages of the document
embeddings_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska-Embeddings.json"

## local destination of all above local required files
data_dir = "./data"

In [6]:
# download data
import os
import requests


def download_pdf_data(pdf_url: str, file_name: str) -> None:
    response = requests.get(pdf_url, stream=True)
    with open(os.path.join(data_dir, file_name), "wb") as file:
        for block in response.iter_content(chunk_size=1024):
            if block:
                file.write(block)


download_pdf_data(pdf_url, file_name)

In [8]:
# prepare data

import fitz
import json


def extract_pdf_text(file_name, file_json):
    document = fitz.open(os.path.join(data_dir, file_name))
    pages = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        pages.append({"page_num": page_num, "text": page_text})

    with open(os.path.join(data_dir, file_json), "w", encoding="utf-8") as file:
        json.dump(pages, file, indent=4, ensure_ascii=False)


extract_pdf_text(file_name, file_json)

In [11]:
# vectorize data
import torch
import numpy as np
from sentence_transformers import SentenceTransformer


def generate_embeddings(file_json, embeddings_json, model):
    pages = []
    with open(os.path.join(data_dir, file_json), "r", encoding="utf-8") as file:
        data = json.load(file)

    for page in data:
        pages.append(page["text"])

    embeddings = model.encode(pages)

    embeddings_paginated = []
    for page_num in range(len(embeddings)):
        embeddings_paginated.append(
            {"page_num": page_num, "embedding": embeddings[page_num].tolist()}
        )

    with open(os.path.join(data_dir, embeddings_json), "w", encoding="utf-8") as file:
        json.dump(embeddings_paginated, file, indent=4, ensure_ascii=False)


model_name = "ipipan/silver-retriever-base-v1.1"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)
generate_embeddings(file_json, embeddings_json, model)

In [13]:
def insert_embeddings(file_json, embeddings_json, client=milvus_client):
    rows = []
    with open(os.path.join(data_dir, file_json), "r", encoding="utf-8") as t_f, open(
        os.path.join(data_dir, embeddings_json), "r", encoding="utf-8"
    ) as e_f:
        text_data, embedding_data = json.load(t_f), json.load(e_f)
        text_data = list(map(lambda d: d["text"], text_data))
        embedding_data = list(map(lambda d: d["embedding"], embedding_data))

        for page, (text, embedding) in enumerate(zip(text_data, embedding_data)):
            rows.append({"text": text, "embedding": embedding})

    client.insert(collection_name="rag_texts_and_embeddings", data=rows)


insert_embeddings(file_json, embeddings_json)

# load inserted data into memory
milvus_client.load_collection("rag_texts_and_embeddings")

In [14]:
# search
def search(model, query, client=milvus_client):
    embedded_query = model.encode(query).tolist()
    result = client.search(
        collection_name="rag_texts_and_embeddings",
        data=[embedded_query],
        limit=1,
        search_params={"metric_type": "L2"},
        output_fields=["text"],
    )
    return result


result = search(model, query="Czym jest sztuczna inteligencja")

In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
import os
from google import genai

GEMINI_KEY = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_KEY)

MODEL = "gemini-flash-latest"


def generate_response(prompt: str):
    try:
        # Send request to Gemini 2.0 Flash API and get the response
        response = gemini_client.models.generate_content(
            model=MODEL,
            contents=prompt,
        )
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [ ]:
def build_prompt(context: str, query: str) -> str:
    prompt = (
        f"Korzystając z kontekstu: {context} Odpowiedz na zadane polecenie: {query}"
    )
    return prompt


def rag(model, query: str) -> str:
    result = search(model, query)
    context = result[0][0].text
    prompt = build_prompt(context, query)
    response = generate_response(prompt)

    return response

In [27]:
rag(model, "Czym jest sztuczna inteligencja?")

'Na podstawie dostarczonego kontekstu, sztuczna inteligencja (AI) jest definiowana w następujący sposób:\n\n1. **W ujęciu potocznym i symbolicznym:** Sztuczna inteligencja to coś (program, maszyna) symulującego inteligencję naturalną, ludzką, będącego wytworem mającym naśladować coś naturalnego.\n\n2. **Jako obszar informatyki:** AI to obszar informatyki, który skupia się na tworzeniu programów komputerowych zdolnych do wykonywania zadań, które wymagają ludzkiej inteligencji.\n\n3. **Główny cel i funkcje:**\n    * Głównym celem AI jest stworzenie systemów, które są zdolne do **myślenia i podejmowania decyzji na sposób przypominający ludzki**.\n    * Realizuje ona niektóre funkcje umysłu, takie jak: rozpoznawanie wzorców, rozumienie języka naturalnego, podejmowanie decyzji, uczenie się, planowanie, komunikowanie, rozwiązywanie problemów oraz dostosowywanie się do sytuacji.\n    * Systemy AI mogą czasami realizować te funkcje w sposób przewyższający funkcje naturalne (np. być wolne od po

In [28]:
rag(model, "Kiedy powastała sztuczna inteligencja?")

'Sztuczna inteligencja (AI) powstała w **latach 50. XX wieku**, kiedy to powstały pierwsze koncepcje i modele tego, co mogłoby się nią stać.\n\nZa jednego z pionierów uznaje się Alana Turinga. (Prawdziwy przełom w tej dziedzinie nastąpił jednak dopiero w latach 80. i 90. XX wieku).'

In [29]:
rag(model, "Kto jest uważany za ojca sztucznej inteligencji?")

'Zgodnie z podanym kontekstem, **Alan Turing** jest wymieniony jako jeden z pionierów sztucznej inteligencji, który sformułował test Turinga, mający na celu ocenę zdolności maszyny do inteligentnego zachowania na poziomie ludzkim.\n\n(Chociaż kontekst nie używa bezpośrednio określenia „ojciec AI”, Alan Turing jest jedynym pionierem wymienionym w sekcji poświęconej historii i powstaniu pierwszych koncepcji).'